In [5]:
import cv2
import os
from imageio.v2 import imread
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import shutil
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [10]:
data_folders = ["Cyst", "Stone", "Tumor", "Normal"]
flat_data_arr = []
target_arr = []
base_dir = 'Dataset'

DATA SPLITTING

In [13]:
# Define the paths for your data
data_dir = 'Dataset'
train_dir = 'TRAIN'
test_dir = 'TEST'
val_dir = 'VAL'

# Define your categories or classes
categories = ["Cyst", "Stone", "Tumor", "Normal"]

# Create train, test, and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split the data into train, test, and validation sets
for category in categories:
    category_dir = os.path.join(data_dir, category)
    files = os.listdir(category_dir)
    train_files, test_val_files = train_test_split(files, test_size=0.3, random_state=42)
    test_files, val_files = train_test_split(test_val_files, test_size=0.5, random_state=42)

    # Move files to appropriate directories
    for file in train_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(train_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for file in test_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(test_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for file in val_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(val_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)


In [6]:
#To get count of data
def count_files(directory):
    print(directory)
    counts = {}
    for category in os.listdir(directory):
        category_dir = os.path.join(directory, category)
        counts[category] = len(os.listdir(category_dir))
    return counts

train_dir = 'TRAIN'
test_dir = 'TEST'
val_dir = 'VAL'

train_counts = count_files(train_dir)
test_counts = count_files(test_dir)
val_counts = count_files(val_dir)

print("Train set:")
print(train_counts)
print("\nTest set:")
print(test_counts)
print("\nValidation set:")
print(val_counts)

#Cyst Stone Tumor Normal
categories = ['Training', 'Testing', 'Validation']
train = list(train_counts.values())
test = list(test_counts.values())
val = list(val_counts.values())

cyst=[train[0],test[0],val[0]]
stone=[train[1],test[1],val[1]]
tumor=[train[2],test[2],val[2]]
normal=[train[3],test[3],val[3]]

trace1 = go.Bar(
   x = categories,
   y = cyst,
   name = 'Cyst',
   marker=dict(color='rgb(26, 118, 255)')
)
trace2 = go.Bar(
   x = categories,
   y = stone,
   name = 'Stone',
   marker=dict(color='rgb(55, 83, 109)')
)
trace3 = go.Bar(
   x = categories,
   y = tumor,
   name = 'Tumor',
   marker=dict(color='rgb(0, 204, 150)')
)
trace4 = go.Bar(
   x = categories,
   y = normal,
   name = 'Normal',
   marker=dict(color='rgb(255, 153, 50)')
)
data = [trace1, trace2, trace3,trace4]
layout = go.Layout(barmode = 'group')
fig = go.Figure(data = data, layout = layout)
go.Figure(fig).show()

TRAIN
TEST
VAL
Train set:
{'Cyst': 2596, 'Normal': 3553, 'Stone': 963, 'Tumor': 1598}

Test set:
{'Cyst': 556, 'Normal': 763, 'Stone': 207, 'Tumor': 342}

Validation set:
{'Cyst': 557, 'Normal': 762, 'Stone': 207, 'Tumor': 343}


In [ ]:
import os


def count_files(directory):
    counts = {}
    for category in os.listdir(directory):
        category_dir = os.path.join(directory, category)
        counts[category] = len(os.listdir(category_dir))
    return counts

train_counts = count_files(train_dir)
test_counts = count_files(test_dir)
val_counts = count_files(val_dir)

print("Train set:")
print(train_counts)
print("\nTest set:")
print(test_counts)
print("\nValidation set:")
print(val_counts)

#Cyst Stone Tumor Normal
categories = ['Training', 'Testing', 'Validation']
train = list(train_counts.values())
test = list(test_counts.values())
val = list(val_counts.values())

cyst=[train[0],test[0],val[0]]
stone=[train[1],test[1],val[1]]
tumor=[train[2],test[2],val[2]]
normal=[train[3],test[3],val[3]]

trace1 = go.Bar(
   x = categories,
   y = cyst,
   name = 'Cyst',
   marker=dict(color='rgb(26, 118, 255)')
)
trace2 = go.Bar(
   x = categories,
   y = stone,
   name = 'Stone',
   marker=dict(color='rgb(55, 83, 109)')
)
trace3 = go.Bar(
   x = categories,
   y = tumor,
   name = 'Tumor',
   marker=dict(color='rgb(0, 204, 150)')
)
trace4 = go.Bar(
   x = categories,
   y = normal,
   name = 'Normal',
   marker=dict(color='rgb(255, 153, 50)')
)
data = [trace1, trace2, trace3,trace4]
layout = go.Layout(barmode = 'group')
fig = go.Figure(data = data, layout = layout)
go.Figure(fig).show()

DATA PREPROCESSING CLAHE,GAUSSIAN_BLUR,CONTOUR DETECTION(cv2.findContour),WATERSHED_SEGMENTATION

In [15]:
def clahe(image_path):
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gray_image)
    return clahe_image

def gaussian_blur(image):
    #image = cv2.imread(image_path)
    kernel_size = (5, 5)
    sigma = 0
    filtered_image = cv2.GaussianBlur(image, kernel_size, sigma)
    return filtered_image

def find_contour(image):
# Preprocessing
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    _, binary_image = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Contour detection
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area or other criteria if necessary
    # For example, you can iterate through contours and filter based on contour area:
    # filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Draw contours on original image
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
    contour_image = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 2)
    return contour_image



def watershed_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labeling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Mark the unknown region with zero
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    markers = cv2.watershed(image, markers)

    # Overlay segmentation result on original image
    image[markers == -1] = [255, 0, 0]  # Mark watershed boundaries

    return image



directories = ['TRAIN','TEST','VAL']  # List of base directories

for directory in directories:
    for folder in data_folders:
        folder_path = os.path.join(directory, folder)
        print("Folder Path:", folder_path)
        
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
                image_path = os.path.join(folder_path, filename)
                
                # Preprocess image
                preprocessed_image = clahe(image_path)
                preprocessed_image = gaussian_blur(preprocessed_image)
                preprocessed_image = find_contour(preprocessed_image)
                preprocessed_image = watershed_segmentation(preprocessed_image)

                # Save preprocessed image in the same location
                output_path = os.path.join(folder_path, filename)
                cv2.imwrite(output_path, preprocessed_image)

print('Preprocess Completed')
print('Preprocess Completed')

Folder Path: TRAIN\Cyst
Folder Path: TRAIN\Stone
Folder Path: TRAIN\Tumor
Folder Path: TRAIN\Normal
Folder Path: TEST\Cyst
Folder Path: TEST\Stone
Folder Path: TEST\Tumor
Folder Path: TEST\Normal
Folder Path: VAL\Cyst
Folder Path: VAL\Stone
Folder Path: VAL\Tumor
Folder Path: VAL\Normal
Preprocess Completed
Preprocess Completed
